# Behandling 1

In [1]:
import requests 
import json
import time 
import tqdm
import pandas as pd

df = pd.read_csv('Boliga Scraped - Rådata.csv')
pd.options.display.max_columns = 100
pd.options.display.max_rows = 500
df.sort_values(by = ['municipality'], axis = 0 );

In [2]:
df.head()

,agentDisplayName,agentRegId,area,basementSize,buildYear,city,createdDate,daysForSale,domainId,energyClass,exp,floor,groupKey,guid,id,images,inWatchlist,isActive,isForeclosure,isPremiumAgent,latitude,longitude,lotSize,municipality,net,openHouse,price,priceChangePercentTotal,propertyType,rooms,selfsale,size,squaremeterPrice,street,views,zipCode,coordinate
0,NaN,24254,1,0,1767,København K,2019-05-22T00:35:07.000Z,93,944,-,16872,3.0,NaN,147A17D5-24CA-4788-8009-F53DEFF2836C,1559151,"[{'id': 1559151, 'date': '2019-08-23T14:46:45....",False,True,False,False,55.67996,12.58478,0,101,67026,NaN,18750000,0,3,6.0,False,280,66964.0,"Kongens Nytorv 21, 3. tv",557,1050,"55.679959999999994, 12.58478"
1,NaN,207,1,88,1880,København K,2019-06-14T03:27:01.490Z,70,7,C,5702,NaN,1.0,EE6F79CE-E04F-4D59-9845-E08F98251FC9,1567291,"[{'id': 1567291, 'date': '2019-08-23T14:46:45....",False,True,False,False,55.67926,12.59149,0,101,0,NaN,10500000,0,3,4.0,False,83,126506.0,"Nyhavn 38, st. Dør/lejl. 4",139,1051,"55.67926, 12.59149"
2,NaN,117,1,0,1755,København K,2019-04-24T00:09:51.880Z,121,171,C,9486,1.0,NaN,23F2A968-9A3B-44C0-BB55-22A9005DEC2C,1549766,"[{'id': 1549766, 'date': '2019-08-23T14:46:45....",False,True,False,False,55.67981,12.59206,0,101,0,NaN,15000000,0,3,7.0,False,274,54744.0,"Nyhavn 53, 1.",110,1051,"55.679809999999996, 12.59206"
3,NaN,17151,1,0,1880,København K,2019-05-08T00:04:30.737Z,107,91,C,17609,5.0,1.0,0CB9E79E-C298-47B9-88E4-23F67F0A0A5B,1554276,"[{'id': 1554276, 'date': '2019-08-23T14:46:45....",False,True,False,False,55.67926,12.59149,0,101,70224,2019-08-25T14:00:00.000Z,19995000,0,3,7.0,False,296,67550.0,"Nyhavn 38, 5. tv.",317,1051,"55.67926, 12.59149"
4,NaN,17489,1,0,1873,København K,2019-07-12T03:21:26.060Z,42,9,C,5590,3.0,NaN,A7380CBF-7220-447E-878E-5B044910369E,1576691,"[{'id': 1576691, 'date': '2019-08-23T14:46:45....",False,True,False,False,55.67852,12.59114,0,101,0,NaN,8750000,0,3,7.0,False,163,53680.0,"Herluf Trolles Gade 22, 3. th",122,1052,"55.67851999999999, 12.59114"


In [55]:
#Oprydning
df.drop(df.loc[df['municipality']==0].index, inplace = True)
df.drop(df.loc[df['price']<=100].index, inplace = True)
df.drop(df.loc[df['latitude']==0].index, inplace = True)
df.drop(df.loc[df['longitude']==0].index, inplace = True)

#Unique municipality df
kommune = df['municipality'].unique()
df_kom = pd.DataFrame(kommune, columns = ['Kommunenr'])

In [61]:
import scraping_class
logfile = 'logfile_boliga.csv'
connector = scraping_class.Connector(logfile)

import time
def kommune_func(kommune_kode):
    url = lambda q: 'https://dawa.aws.dk/kommuner/{}'.format(q)
    resp, call_id = connector.get(url(kommune_kode), 'Kommunedata')
    time.sleep(0.5)
    
    if not resp.status_code == 200:
        return ''
    resp = resp.json()
    kommune = resp['navn']
    
    return kommune

In [69]:
import scraping_class
logfile = 'Boligscrape_log.csv'
connector = scraping_class.Connector(logfile)

import time
def region_func(kommune_kode):
    url = lambda q: 'https://dawa.aws.dk/kommuner/{}'.format(q)
    resp, call_id = connector.get(url(kommune_kode), 'Kommunedata')
    time.sleep(0.5)
    
    if not resp.status_code == 200:
        return ''
    resp = resp.json()
    region = resp['region']['navn']
    
    return region

In [63]:
df_kom['Kommune'] = df_kom.Kommunenr.apply(kommune_func)

In [70]:
df_kom['Region'] = df_kom.Kommunenr.apply(region_func)

In [72]:
df = df.merge(df_kom, how='left', left_on=['municipality'],
                           right_on=['Kommunenr'], validate = 'm:1')

df = df.drop(columns = ['Kommunenr'], axis = 1)

In [139]:
# Convert Prop_type (number) to descriptive labels
prop_type = {'Prop_Type': {'Villa': 1, 'Rækkehus': 2, 'Ejerlejlighed': 3,
            'Fritidshus': 4, 'Andelsbolig': 5, 'Landejendom': 6,
            'Helårsgrund': 7, 'Fritidsgrund': 8, 'Villalejlighed': 9}}

prop_type_df = pd.DataFrame(prop_type).reset_index()

df = df.merge(prop_type_df, how='left', left_on=['propertyType'],
                           right_on=['Prop_Type'], validate = 'm:1')

df = df.drop(columns = ['Prop_Type'], axis = 1)
df.rename(columns={'index':'Type'}, inplace=True)
#Drop missing in Type
df = df[df.Type.isnull() != True]

In [6]:
df.to_csv('Boliga - Behandling 1 - Kom, Reg, Type.csv', index = False)